In [1]:
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
import os
import requests


/usr/local/Ascend/ascend-toolkit/7.0.RC1/python/site-packages/tbe/tvm/contrib/ccec.py:766: DeprecationWarning: invalid escape sequence \L
  if not dirpath.find("AppData\Local\Temp"):
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/classifier/transdata/transdata_classifier.py:222: DeprecationWarning: invalid escape sequence \B
  """
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/vector/transdata/common/graph/transdata_graph_info.py:140: DeprecationWarning: invalid escape sequence \c
  """
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, 

In [3]:
# 加载数据集

with open('sales_textbook.txt', 'r', encoding='utf-8') as f:
    text = f.read()
# text

In [4]:
# 超参数
context_length = 16
d_model = 64
batch_size  = 4
num_heads = 4

In [ ]:
!pip install tiktoken

In [6]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")

In [7]:
tokenized_text = encoding.encode(text)
len(tokenized_text)

77919

In [8]:
import mindspore
from mindspore import Tensor
import mindspore.ops as ops
import numpy as np

tokenized_text = Tensor(np.array(tokenized_text), mindspore.int32)
max_token_value = ops.ReduceMax()(tokenized_text).asnumpy().item()

[WARNING] DEVICE(7331,fffed8fcd1e0,python):2025-05-19-21:39:21.025.281 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_runtime_manager.cc:46] GetAscendRuntime] No ascend runtime creator for AscendVM with device id 0
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/extract_image_patches_without_cbuf_schedule.py:317: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if _ is not 1:
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/extract_image_patches_without_cbuf_schedule.py:317: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if _ is not 1:
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/extract_image_patches_without_cbuf_schedule.py:317: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if _ is not 1:
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/extract_image_patches_without_cbuf_schedule.py:317: Synt

In [9]:
train_index = int(tokenized_text.shape[0] * 0.9)
train_data = tokenized_text[:train_index]
val_data = tokenized_text[train_index:]


In [54]:
# x 是输入，y 是模型应该输出的正确答案。我们用 x 输入模型，计算输出再和 y 做对比，计算 loss（损失函数），从而训练模型。|
data = train_data

# 随机索引
idxs = np.random.randint(0, data.shape[0] - context_length, size=(batch_size,))
# 构造 x_batch 和 y_batch
x_list = []
y_list = []

for i in idxs:
    i = int(i)
    x = data[i:i+context_length]
    y = data[i+1:i+context_length+1]
    x_list.append(x)
    y_list.append(y)

# 堆叠成 batch 维度
x_batch = ops.Stack()(x_list)
y_batch = ops.Stack()(y_list)

# 查看形状
print(x_batch.shape)

(4, 16)


In [55]:
import pandas as pd

# 将 Tensor 转成 NumPy 并展示为表格
pd.DataFrame(x_batch[0].asnumpy())

,0
0,2532
1,13
2,2435
3,3493
4,14509
5,6029
6,430
7,701
8,10209
9,28421


In [56]:
# 解码为文本（假设 encoding 是同一个 tokenizer）
encoding.decode(x_batch[0].asnumpy())

' service. They provide concrete evidence that your offering delivers on its promises.\nTo effectively'

In [57]:
import mindspore.nn as nn

# 定义嵌入层：词表大小是 max_token_value + 1，输出维度是 d_model
input_embedding_lookup_table = nn.Embedding(vocab_size=max_token_value+1, embedding_size=d_model)

In [58]:
# 执行嵌入查找
x_batch_embedded = input_embedding_lookup_table(x_batch)
y_batch_embedded = input_embedding_lookup_table(y_batch)
x_batch_embedded.shape

(4, 16, 64)

In [59]:
import math
import mindspore
import mindspore.numpy as msnp
from mindspore import Tensor

# 创建位置编码张量 shape: (context_length, d_model)
position_encoding_lookup_table = msnp.zeros((context_length, d_model), dtype=mindspore.float32)

# 生成位置索引 shape: (context_length, 1)
position = msnp.arange(context_length, dtype=mindspore.float32).reshape((-1, 1))

# 生成 div_term: (d_model//2,)
div_term = msnp.exp(msnp.arange(0, d_model, 2, dtype=mindspore.float32) * (-math.log(10000.0) / d_model))

# 计算 sin 和 cos，填入对应位置
position_encoding_lookup_table[:, 0::2] = msnp.sin(position * div_term)
position_encoding_lookup_table[:, 1::2] = msnp.cos(position * div_term)

# 扩展到 batch 维度，shape: (batch_size, context_length, d_model)
position_encoding_lookup_table = msnp.broadcast_to(position_encoding_lookup_table[None, :, :], (batch_size, context_length, d_model))


In [60]:
x = x_batch_embedded + position_encoding_lookup_table
y = y_batch_embedded + position_encoding_lookup_table
pd.DataFrame(x[0].asnumpy())

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-0.010492,0.984643,0.009938,0.991088,-0.002256,1.003334,-0.000210,1.002060,-0.011284,0.979168,...,0.009664,0.998589,-0.000421,0.981606,0.010288,0.989051,0.008825,0.994384,-0.022240,0.999825
1,0.830274,0.539474,0.701702,0.718858,0.524197,0.850354,0.400135,0.915532,0.306686,0.941032,...,0.010515,1.004618,0.004719,1.014514,-0.015299,1.000234,0.011600,1.003430,-0.007476,0.998228
2,0.905793,-0.434523,1.003516,0.068699,0.923278,0.441271,0.765724,0.666008,0.589084,0.827341,...,0.006229,0.996058,0.003137,1.009670,-0.002933,1.000713,0.000297,0.988314,0.015004,0.991415
3,0.163966,-0.992298,0.782453,-0.634649,1.010733,-0.098078,0.956213,0.308620,0.818634,0.581636,...,-0.015616,0.999082,0.005030,1.003834,0.006453,0.990989,0.001307,0.996752,0.009666,1.005885
4,-0.772759,-0.645629,0.136062,-0.992607,0.784563,-0.628715,1.002363,-0.115549,0.939434,0.293704,...,0.002499,1.007297,0.003115,0.991969,0.009228,0.993695,0.000849,1.000376,0.009866,0.996763
5,-0.962862,0.285432,-0.554096,-0.820274,0.308784,-0.950760,0.865867,-0.493208,1.004814,-0.009990,...,0.000568,0.999633,-0.013092,1.000590,0.000782,1.002231,0.004664,1.007651,-0.014741,0.990456
6,-0.264728,0.960901,-0.989609,-0.206996,-0.218059,-0.964465,0.569617,-0.803106,0.929052,-0.331796,...,0.002515,0.999593,0.013324,0.991281,-0.007830,0.995255,0.005118,1.008260,0.001472,1.011064
7,0.650735,0.757812,-0.855529,0.525213,-0.709279,-0.699080,0.189181,-0.979096,0.791830,-0.592893,...,-0.015753,1.005486,-0.010350,0.982370,0.020715,0.989393,-0.003478,1.009982,-0.006033,1.012123
8,0.992442,-0.162564,-0.285129,0.959738,-0.988017,-0.205889,-0.229851,-0.972330,0.594239,-0.813461,...,-0.000789,0.999781,0.008198,1.012132,0.009522,0.988123,-0.013434,0.988673,0.007803,1.000904
9,0.420256,-0.928315,0.455221,0.897506,-0.964682,0.322646,-0.608655,-0.791092,0.289169,-0.945931,...,0.000923,0.992933,0.005325,0.995589,0.002262,1.011278,0.001482,0.989727,-0.002923,0.976384


In [61]:
import mindspore.nn as nn

# 定义线性层（等价于 PyTorch 的 nn.Linear）
wq = nn.Dense(d_model, d_model)
wk = nn.Dense(d_model, d_model)
wv = nn.Dense(d_model, d_model)

# 执行前向传播，x.shape 应为 (batch_size, context_length, d_model)
Q = wq(x)
K = wk(x)
V = wv(x)

# 打印 Q 的形状
print(Q.shape)


(4, 16, 64)


In [62]:
# 重塑形状并维度重排
Q = Q.reshape((batch_size, context_length, num_heads, d_model // num_heads))
Q = Q.transpose(0, 2, 1, 3)  # (batch, heads, seq, head_dim)

K = K.reshape((batch_size, context_length, num_heads, d_model // num_heads))
K = K.transpose(0, 2, 1, 3)

V = V.reshape((batch_size, context_length, num_heads, d_model // num_heads))
V = V.transpose(0, 2, 1, 3)

# 打印 V 的形状
print(V.shape)


(4, 4, 16, 16)


In [63]:
import math
import mindspore.ops as ops

# Q: (batch, heads, seq_len, head_dim)
# K: (batch, heads, seq_len, head_dim)

# 对 K 做转置，最后两个维度对调
K_T = ops.transpose(K, (0, 1, 3, 2))  # (batch, heads, head_dim, seq_len)

# 执行 batch matmul
scores = ops.matmul(Q, K_T) / math.sqrt(d_model // num_heads)

# 打印注意力得分形状
print(scores.shape)


(4, 4, 16, 16)


In [64]:
# 确认scores形状和类型
import mindspore.numpy as mnp

print("scores shape:", scores.shape, "dtype:", scores.dtype)

# 创建掩码
mask = mnp.triu(mnp.ones((context_length, context_length)), k=1).astype(mindspore.bool_)
mask = mask.expand_dims(0).expand_dims(0)
mask = mask.broadcast_to(scores.shape)

# 创建全 -inf 张量，和 scores 形状相同
neg_inf = mnp.full(scores.shape, float('-inf'), dtype=mindspore.float32)

# 用 Select 替换
masked_scores = ops.Select()(mask, neg_inf, scores)



scores shape: (4, 4, 16, 16) dtype: Float32


In [65]:
# Softmax
softmax = mindspore.nn.Softmax(axis=-1)
attention_score = softmax(masked_scores)

pd.DataFrame(attention_score[0, 0].asnumpy())


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.480087,0.519913,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.304505,0.333644,0.361851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.220615,0.242750,0.264118,0.272517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.178030,0.194621,0.209441,0.213723,0.204184,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.153753,0.166394,0.176249,0.177224,0.167943,0.158436,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.135404,0.145462,0.152393,0.151830,0.143561,0.136337,0.135012,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.116180,0.125328,0.132003,0.132346,0.126117,0.120798,0.120890,0.126338,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.097684,0.106301,0.113567,0.116129,0.113013,0.109713,0.110297,0.114237,0.119058,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.083840,0.091795,0.099273,0.103706,0.103465,0.101852,0.102125,0.103816,0.105374,0.104753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [66]:
# 矩阵乘法，batch matmul
A = ops.matmul(attention_score, V)  # 形状 (batch_size, num_heads, seq_len, head_dim)

pd.DataFrame(A[0, 0].asnumpy())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.527832,-0.750488,0.930176,-0.086548,0.086487,0.211670,0.165283,-0.182983,-0.791992,0.258545,-0.294189,-0.528809,-0.013718,-0.126831,0.201172,0.397461
1,0.573791,-0.828948,0.952774,-0.060061,0.117433,0.207290,0.177090,-0.117981,-0.771425,0.198875,-0.369602,-0.491864,-0.164647,-0.013489,0.183398,0.433517
2,0.557681,-0.848720,0.978776,0.015252,0.149786,0.200909,0.174053,-0.058563,-0.738512,0.121293,-0.409987,-0.448211,-0.305077,0.124830,0.180136,0.443745
3,0.501189,-0.809438,1.013115,0.106501,0.158509,0.202291,0.167276,-0.037188,-0.703624,0.053205,-0.405880,-0.403906,-0.390875,0.239345,0.189835,0.416159
4,0.440036,-0.734977,1.048582,0.177242,0.138789,0.197522,0.163208,-0.055578,-0.669305,0.016024,-0.375793,-0.360252,-0.410505,0.301418,0.197313,0.365889
5,0.396364,-0.662066,1.083122,0.216677,0.106631,0.191787,0.164581,-0.087499,-0.640053,0.009201,-0.348469,-0.321966,-0.398996,0.325731,0.203175,0.308102
6,0.366939,-0.598603,1.114421,0.237537,0.080981,0.184585,0.166691,-0.115859,-0.609161,0.023580,-0.340636,-0.286821,-0.389177,0.332927,0.207571,0.253294
7,0.344945,-0.541024,1.146114,0.261622,0.071457,0.176405,0.164098,-0.128623,-0.581024,0.048731,-0.362588,-0.250471,-0.400802,0.343207,0.218491,0.206874
8,0.325145,-0.488687,1.173580,0.299846,0.077379,0.175176,0.153134,-0.122777,-0.558095,0.073265,-0.404773,-0.216540,-0.435119,0.357994,0.241077,0.173257
9,0.309089,-0.444724,1.187039,0.343548,0.086899,0.187274,0.140149,-0.111964,-0.541677,0.087444,-0.443830,-0.191434,-0.470298,0.372022,0.270983,0.157047


In [67]:
# 先transpose，将 num_heads 和 seq_len 维度交换
A = ops.transpose(A, (0, 2, 1, 3))  # 变成 (batch_size, seq_len, num_heads, head_dim)

# 再reshape，将最后两维合并成 d_model (num_heads * head_dim)
A = A.reshape(batch_size, -1, d_model)  # 形状 (batch_size, seq_len, d_model)

print(A.shape)


(4, 16, 64)


In [68]:
wo = nn.Dense(d_model,d_model)
output = wo(A)
output = output + x
output.shape

(4, 16, 64)

In [69]:
layer_norm = nn.LayerNorm([d_model])  # 注意：需要用列表形式指定 normalized_shape
layer_norm_output = layer_norm(output)

print(layer_norm_output.shape)

(4, 16, 64)


In [70]:
# 前馈层
output = nn.Dense(d_model, d_model * 4)(layer_norm_output)
output = nn.ReLU()(output)
output = nn.Dense(d_model * 4, d_model)(output)
output.shape



(4, 16, 64)

In [71]:
output = output + layer_norm_output
output.shape

(4, 16, 64)

In [72]:
# 再进行一次layernorm
output = layer_norm(output)
output.shape

(4, 16, 64)

In [73]:
# 假设你已经定义了 d_model 和 max_token_value
final_linear_layer = nn.Dense(d_model, max_token_value + 1)

# output 是形状为 (batch_size, seq_len, d_model) 的张量
output = final_linear_layer(output)
output.shape

(4, 16, 100070)

In [74]:
from mindspore import nn, ops

# 1. softmax（对最后一维）
softmax = nn.Softmax(axis=-1)
final_output = softmax(output)  # 形状: (batch_size, seq_len, vocab_size)

# 2. 取第一个 token 的预测结果（如 batch[0], position[0]）
predicted_idx = ops.Argmax()(final_output[0, 0]).asnumpy().item()

# 3. 解码为字符
decoded_token = encoding.decode([predicted_idx])
print(decoded_token)


 GENERAL
